In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
atlas = src.graph.graph_from_json('Outputs/atlas.json')

graph_empty = src.graph.graph_from_json('Outputs/graph_empty.json')

In [3]:
# Adding roadnum for destinations

graph_to_atlas, atlas_to_graph = src.adjacency.node_assignment(graph_empty, atlas)

for graph_node, atlas_node in graph_to_atlas.items():

    graph_empty._node[graph_node]['roadnum'] = atlas._node[atlas_node]['roadnum']

In [5]:
# Edge-less graph consisting of locations from graph_empty and nodes which
# represent intersections in atlas

graph_to_atlas, atlas_to_graph = src.adjacency.node_assignment(graph_empty, atlas)

destinations = list(graph_empty.nodes)

destinations_atlas = [graph_to_atlas[node] for node in destinations]

intersections = []

for source, adj in atlas._adj.items():

    if len(adj) > 2:

        intersections.append(source)

In [8]:
len(intersections), len(intersections + destinations_atlas)

(5722, 7890)

In [33]:
deep_reload(src)

objective = src.adjacency.Graph_From_Atlas(
    limits = [np.inf, 786e3, np.inf]
)

reduced_atlas = src.adjacency.reduction(atlas, graph_empty, objective)

 [------>] (7137/7137) 100.00%, 278.09 it/s, 25.67 s elapsed, 0.00 s remaining

In [34]:
reduced_atlas.number_of_nodes(), reduced_atlas.number_of_edges()

(7137, 18215)

In [35]:
k = list(reduced_atlas._adj.keys())
len(k), max(k)

(7137, 7136)

In [39]:
reduced_atlas._adj[0]

{0: {'time': 0, 'distance': 0, 'price': 0},
 17: {'time': 85.5, 'distance': 1710.0, 'price': 0},
 5534: {'time': 257.5625, 'distance': 4850.0, 'price': 0},
 5547: {'time': 225.4090909090909, 'distance': 5510.0, 'price': 0}}

In [37]:
deep_reload(src)

objective = src.adjacency.Graph_From_Atlas(
    limits = [np.inf, 786e3, np.inf]
)

graph = src.adjacency.adjacency(
    reduced_atlas,
    graph_empty,
    objective = objective,
)

 [------>] (2168/2168) 100.00%, 10.35 it/s, 209.59 s elapsed, 0.00 s remaining

In [40]:
src.graph.graph_to_json(graph, 'Outputs/graph_full_red.json')

In [38]:
graph.number_of_nodes(), graph.number_of_edges()

(2168, 2161851)

In [44]:
places = [k for k, v in graph._node.items() if v['type'] == 'place']
stations = [k for k, v in graph._node.items() if v['type'] == 'station']

tesla = [k for k, v in graph._node.items() if (v['type'] == 'station') and (v['network'] == 'Tesla')]
non_tesla = [k for k, v in graph._node.items() if (v['type'] == 'station') and (v['network'] != 'Tesla')]

In [46]:
len(tesla), len(non_tesla), len(places)

(419, 1270, 479)

In [47]:
graph_tesla = src.graph.subgraph(graph, places + tesla)

In [48]:
graph_tesla.number_of_nodes(), graph_tesla.number_of_edges()

(898, 366871)

In [49]:
src.graph.graph_to_json(graph_tesla, 'Outputs/graph_full_tesla.json')

In [50]:
graph_non_tesla = src.graph.subgraph(graph, places + non_tesla)
graph_non_tesla.number_of_nodes(), graph_non_tesla.number_of_edges()

(1749, 1411222)

In [51]:
src.graph.graph_to_json(graph_non_tesla, 'Outputs/graph_full_non_tesla.json')